In [1]:
from Pipeline import Pipeline
import Adapters
import Processors
import Sinks

/home/daniel/anaconda3/envs/KD/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
[nltk_data] Downloading package punkt to /home/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the check

In [2]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="aifb-ls3-vm1.aifb.kit.edu", username="", password="", index="document")

### Pipeline Definition

In [3]:
adapter = Adapters.UnarxiveAdapter("/home/daniel/storage/KD/samples")
pipeline = Pipeline(adapter, batch_size=100, cpus=10)

In [4]:
#MetadataArxiveEnricher
from arxive_metadata.rocksDB import RocksDBAdapter
db = RocksDBAdapter("arXive_metadata", "http://localhost:8089")
pipeline.add_processor(Processors.MetadataArxiveEnricher("arixive-id",db))

In [5]:
#FilterOnMetadataValue
cs_values = ["cs.AI", "cs.AR", "cs.CC", "cs.CE", "cs.CG", "cs.CL", "cs.CR","cs.CV","cs.CY","cs.DB","cs.DC","cs.DL","cs.DM","cs.DS","cs.ET","cs.FL","cs.GL","cs.GR", "cs.GT","cs.HC","cs.IR","cs.LG","cs.LO","cs.MA","cs.MM","cs.MS","cs.NA","cs.NE","cs.NI","cs.OH","cs.OS","cs.PF","cs.PL","cs.RO","cs.SC","cs.SD","cs.SE","cs.SI","cs.SY"]
pipeline.add_processor(Processors.FilterOnMetadataValue("categories", cs_values))

In [6]:
#Clean up text
#TextKeywordCut Introduction
pipeline.add_processor(Processors.TextKeywordCut("Introduction"))
#TextKeywordCut Acknowledgements
pipeline.add_processor(Processors.TextKeywordCut("Acknowledgments", False))
#TextAppendMetadataField
pipeline.add_processor(Processors.TextAppendMetadataField("abstract", True))

In [7]:
#MetadataFieldDiscarder
fields_to_discard = ["update_date", "authors", "abstract", "versions", "comments", "report-no", "license", "id", "categories", "submitter", "journal-ref", "authors_parsed"]
pipeline.add_processor(Processors.MetadataFieldDiscarder(fields_to_discard))

In [8]:
#MetadataMagArxiveLinker
import pandas as pd
df = pd.read_csv("../../data/unarxive_metadata/mag_id_2_arxiv_id.csv", names=["mag_id", "idk", "url", "arxive_id"])
pipeline.add_processor(Processors.MetadataMagArxiveLinker(df, column_to_match="arxive_id", column_to_add="mag_id", field_to_match="arixive-id"))
del df

In [9]:
#TextReplaceFilter
pipeline.add_processor(Processors.TextReplaceFilter(r"\{{(.*?)\}}", ""))

In [10]:
#HaystackPreProcessor
from haystack.preprocessor import PreProcessor
pre_processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=False,
    split_overlap=0)
pipeline.add_processor(Processors.HaystackPreProcessor(pre_processor))

In [11]:
#TextfileSink
pipeline.add_sink(Sinks.TextfileSink("/home/daniel/storage/KD/testoutput", ["arixive-id", "_split_id"]))
#pipeline.add_sink(Sinks.TextfileSink("/home/daniel/storage/KD/manual_classification_sample", ["arixive-id"]))

In [ ]:
#ElasticsearchSink
pipeline.add_sink(Sinks.ElasticsearchSink(document_store))

In [ ]:
#PrintSink
pipeline.add_sink(Sinks.PrintSink())

In [14]:
print(pipeline)

(UnarxiveAdapter) ---> *MetadataArxiveEnricher* ---> *FilterOnMetadataValue* ---> *TextKeywordCut* ---> *TextKeywordCut* ---> *TextAppendMetadataField* ---> *MetadataFieldDiscarder* ---> *MetadataMagArxiveLinker* ---> *TextReplaceFilter* ---> *HaystackPreProcessor*
===> |_TextfileSink_|


### Use Pipeline to process documents

In [15]:
pipeline.process()

Processing batches (10 minibatches at once): [############################################################] 21/21


### Pipeline to update entries in Elasticsearch

In [ ]:
e_adapt = Adapters.ElasticsearchAdapter(document_store, batch_size=10)

In [ ]:
update_pipeline = Pipeline(e_adapt, batch_size=20)

In [ ]:
#ElasticsearchSink (put items back into Elasticsearch)
update_document_store = ElasticsearchDocumentStore(update_existing_documents=True, host="localhost", username="", password="", index="document")
update_pipeline.add_sink(Sinks.ElasticsearchSink(update_document_store))

In [ ]:
print(update_pipeline)

In [ ]:
update_pipeline.process()

In [ ]:
len(adapter)

In [ ]:
#IMRaD classification
from imrad_classification import BERTClassificationHandler
classification_handler = BERTClassificationHandler("/home/daniel/BERT_copy.model")
pipeline.add_processor(Processors.IMRaDClassification(classification_handler))